In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
try:
    address = "/content/drive/My Drive/Doctor/Research/CNN/Colab/"
    data_csv = pd.read_csv(address+'Data_Ny.csv', index_col=0)
except:
    address = ""
    data_csv = pd.read_csv(address+'Data_Ny.csv', index_col=0)

In [ ]:
# FNN

M = 18 # The number of past data for input
H = 6 # The number of future data for output
N = 67 # The number of sites

in_size     = N*M
hidden_size = N*M*2
out_size    = N*H

AR_learning_rate = 1e-4; AR_epoch = 150
FNN_learning_rate = 1e-5; FNN_epoch = 150

class AR(nn.Module):
    def __init__(self):
        super(AR, self).__init__()
        self.fc = nn.Linear(in_size, out_size)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=AR_learning_rate)

    def forward(self, x):
        x = self.fc(x)
        return x

    def train_net(self, x, y):
        x, y = torch.tensor(x,dtype=torch.float), torch.tensor(y,dtype=torch.float)
        loss = F.mse_loss(self.forward(x), y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

class FNN(nn.Module):
    def __init__(self):
        super(FNN, self).__init__()
        self.layer1 = nn.Linear(in_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, out_size)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=FNN_learning_rate)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.fc_out(x)
        return x

    def train_net(self, x, y):
        x, y = torch.tensor(x,dtype=torch.float), torch.tensor(y,dtype=torch.float)
        loss = F.mse_loss(self.forward(x), y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [ ]:
# Data

data = np.array(data_csv)
data_size = np.size(data, 1)

train_data_size  = int(data_size*0.6)
train_batch_size = train_data_size - M - H + 1

batch_size = 128
total_batch = int((train_batch_size-1)/batch_size) + 1

train_input  = np.zeros((train_batch_size, N*M))
train_output = np.zeros((train_batch_size, N*H))
for i in range(train_batch_size):
    train_input[i,:]  = np.reshape(data[:,i:i+M], (N*M))
    train_output[i,:] = np.reshape(data[:,i+M:i+M+H], (N*H))

val_data_size    = int(data_size*0.8)
val_batch_size   = val_data_size - train_data_size - M - H + 1

val_input  = np.zeros((val_batch_size, N*M))
val_output = np.zeros((val_batch_size, N*H))
for i in range(val_batch_size):
    val_input[i,:]  = np.reshape(data[:,train_data_size+i:train_data_size+i+M], (N*M))
    val_output[i,:] = np.reshape(data[:,train_data_size+i+M:train_data_size+i+M+H], (N*H))

test_batch_size   = data_size - val_data_size - M - H + 1

test_input  = np.zeros((test_batch_size, N*M))
test_output = np.zeros((test_batch_size, N, H))
for i in range(test_batch_size):
    test_input[i,:]  = np.reshape(data[:,val_data_size+i:val_data_size+i+M], (N*M))
    test_output[i,:] = np.reshape(data[:,val_data_size+i+M:val_data_size+i+M+H], (N, H))

In [ ]:
# AR training

model = AR()
mse_train, mse_val = [], [] # Mean Squared Error
mae_train, mae_val = [], [] # Mean Absolute Error
for epoch in range(AR_epoch):
    for i in range(total_batch):
        batch_x = train_input[batch_size*i:batch_size*(i+1)]
        batch_y = train_output[batch_size*i:batch_size*(i+1)]
        model.train_net(batch_x, batch_y)
    
    train_predict = model.forward(torch.tensor(train_input, dtype=torch.float)).detach().numpy()
    mse_train = np.mean(np.square(train_predict - train_output))
    mae_train = np.mean(np.abs(train_predict - train_output))
    scale_train = np.mean(np.abs(train_output[1:] - train_output[:-1]))
    
    val_predict = model.forward(torch.tensor(val_input, dtype=torch.float)).detach().numpy()
    mse_val = np.mean(np.square(val_predict - val_output))
    mae_val = np.mean(np.abs(val_predict - val_output))
    scale_val = np.mean(np.abs(val_output[1:] - val_output[:-1]))

    NRMSE_train = round(100*np.sqrt(mse_train),2)
    MAPE_train  = round(100*mae_train,2)
    MASE_train  = round(mae_train/scale_train,2)
    NRMSE_val   = round(100*np.sqrt(mse_val),2)
    MAPE_val    = round(100*mae_val,2)
    MASE_val    = round(mae_val/scale_val,2)

    if epoch == 0 or (epoch+1) % 10 == 0:
        print("epoch: {}".format(epoch+1))
        print("NRMSE_train: {}%".format(NRMSE_train).ljust(25), end="")
        print("MAPE_train: {}%".format(MAPE_train).ljust(25), end="")
        print("MASE_train: {}".format(MASE_train).ljust(25))
        print("NRMSE_val: {}%".format(NRMSE_val).ljust(25), end="")
        print("MAPE_val: {}%".format(MAPE_val).ljust(25), end="")
        print("MASE_val: {}".format(MASE_val).ljust(25))
        print("------------------------------------------------------------------------------------------")

epoch: 1
NRMSE_train: 16.63%      MAPE_train: 12.68%       MASE_train: 2.59         
NRMSE_val: 15.81%        MAPE_val: 11.89%         MASE_val: 2.67           
------------------------------------------------------------------------------------------
epoch: 10
NRMSE_train: 11.33%      MAPE_train: 8.17%        MASE_train: 1.67         
NRMSE_val: 10.42%        MAPE_val: 7.46%          MASE_val: 1.67           
------------------------------------------------------------------------------------------
epoch: 20
NRMSE_train: 10.68%      MAPE_train: 7.62%        MASE_train: 1.55         
NRMSE_val: 9.97%         MAPE_val: 7.1%           MASE_val: 1.59           
------------------------------------------------------------------------------------------
epoch: 30
NRMSE_train: 10.35%      MAPE_train: 7.32%        MASE_train: 1.49         
NRMSE_val: 9.76%         MAPE_val: 6.94%          MASE_val: 1.56           
--------------------------------------------------------------------------------

In [ ]:
# AR test

test_predict = model.forward(torch.tensor(test_input, dtype=torch.float)).detach().numpy()
test_predict = np.reshape(test_predict, (-1, N, H))
mse_test = np.mean(np.square(test_predict - test_output))
mae_test = np.mean(np.abs(test_predict - test_output))
scale_test = np.mean(np.abs(test_output[1:] - test_output[:-1]))

NRMSE_test = round(100*np.sqrt(mse_test),2)
MAPE_test  = round(100*mae_test,2)
MASE_test  = round(mae_test/scale_test,2)

print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
print("MAPE_test: {}%".format(MAPE_test).ljust(25), end="")
print("MASE_test: {}".format(MASE_test).ljust(25))

test_output_a = np.mean(test_output, axis=1)
test_predict_a = np.mean(test_predict, axis=1)
mae_test_a = np.mean(np.abs(test_predict_a - test_output_a))
scale_test_a = np.mean(np.abs(test_output_a[1:] - test_output_a[:-1]))

MAPE_test_a      = round(100*mae_test_a,2)
MAPE_improvement = round(100*(MAPE_test-MAPE_test_a)/MAPE_test,2)

print("MAPE_test (Aggregation): {}%".format(MAPE_test_a).ljust(36), end="")
print("MAPE_test Improvement: {}%".format(MAPE_improvement).ljust(36))

NRMSE_test: 9.99%        MAPE_test: 7.55%         MASE_test: 2.36          
MAPE_test (Aggregation): 6.53%      MAPE_test Improvement: 13.51%       


In [ ]:
# FNN training

model = FNN()
mse_train, mse_val = [], [] # Mean Squared Error
mae_train, mae_val = [], [] # Mean Absolute Error
for epoch in range(FNN_epoch):
    for i in range(total_batch):
        batch_x = train_input[batch_size*i:batch_size*(i+1)]
        batch_y = train_output[batch_size*i:batch_size*(i+1)]
        model.train_net(batch_x, batch_y)
    
    train_predict = model.forward(torch.tensor(train_input, dtype=torch.float)).detach().numpy()
    mse_train = np.mean(np.square(train_predict - train_output))
    mae_train = np.mean(np.abs(train_predict - train_output))
    scale_train = np.mean(np.abs(train_output[1:] - train_output[:-1]))
    
    val_predict = model.forward(torch.tensor(val_input, dtype=torch.float)).detach().numpy()
    mse_val = np.mean(np.square(val_predict - val_output))
    mae_val = np.mean(np.abs(val_predict - val_output))
    scale_val = np.mean(np.abs(val_output[1:] - val_output[:-1]))

    NRMSE_train = round(100*np.sqrt(mse_train),2)
    MAPE_train  = round(100*mae_train,2)
    MASE_train  = round(mae_train/scale_train,2)
    NRMSE_val   = round(100*np.sqrt(mse_val),2)
    MAPE_val    = round(100*mae_val,2)
    MASE_val    = round(mae_val/scale_val,2)

    if epoch == 0 or (epoch+1) % 10 == 0:
        print("epoch: {}".format(epoch+1))
        print("NRMSE_train: {}%".format(NRMSE_train).ljust(25), end="")
        print("MAPE_train: {}%".format(MAPE_train).ljust(25), end="")
        print("MASE_train: {}".format(MASE_train).ljust(25))
        print("NRMSE_val: {}%".format(NRMSE_val).ljust(25), end="")
        print("MAPE_val: {}%".format(MAPE_val).ljust(25), end="")
        print("MASE_val: {}".format(MASE_val).ljust(25))
        print("------------------------------------------------------------------------------------------")

epoch: 1
NRMSE_train: 22.92%      MAPE_train: 14.55%       MASE_train: 2.97         
NRMSE_val: 20.85%        MAPE_val: 13.04%         MASE_val: 2.92           
------------------------------------------------------------------------------------------
epoch: 10
NRMSE_train: 10.73%      MAPE_train: 6.56%        MASE_train: 1.34         
NRMSE_val: 10.08%        MAPE_val: 6.32%          MASE_val: 1.42           
------------------------------------------------------------------------------------------
epoch: 20
NRMSE_train: 10.07%      MAPE_train: 6.06%        MASE_train: 1.24         
NRMSE_val: 9.66%         MAPE_val: 5.98%          MASE_val: 1.34           
------------------------------------------------------------------------------------------
epoch: 30
NRMSE_train: 9.68%       MAPE_train: 5.76%        MASE_train: 1.18         
NRMSE_val: 9.51%         MAPE_val: 5.84%          MASE_val: 1.31           
--------------------------------------------------------------------------------

In [ ]:
# FNN test

test_predict = model.forward(torch.tensor(test_input, dtype=torch.float)).detach().numpy()
test_predict = np.reshape(test_predict, (-1, N, H))
mse_test = np.mean(np.square(test_predict - test_output))
mae_test = np.mean(np.abs(test_predict - test_output))
scale_test = np.mean(np.abs(test_output[1:] - test_output[:-1]))

NRMSE_test = round(100*np.sqrt(mse_test),2)
MAPE_test  = round(100*mae_test,2)
MASE_test  = round(mae_test/scale_test,2)

print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
print("MAPE_test: {}%".format(MAPE_test).ljust(25), end="")
print("MASE_test: {}".format(MASE_test).ljust(25))

test_output_a = np.mean(test_output, axis=1)
test_predict_a = np.mean(test_predict, axis=1)
mae_test_a = np.mean(np.abs(test_predict_a - test_output_a))
scale_test_a = np.mean(np.abs(test_output_a[1:] - test_output_a[:-1]))

MAPE_test_a      = round(100*mae_test_a,2)
MAPE_improvement = round(100*(MAPE_test-MAPE_test_a)/MAPE_test,2)

print("MAPE_test (Aggregation): {}%".format(MAPE_test_a).ljust(36), end="")
print("MAPE_test Improvement: {}%".format(MAPE_improvement).ljust(36))

NRMSE_test: 9.63%        MAPE_test: 5.83%         MASE_test: 1.82          
MAPE_test (Aggregation): 4.94%      MAPE_test Improvement: 15.27%       
